# [실습] LangChain 기초

이번 실습에서는 Lang chain에 대해서 학습합니다. OPEN AI의 API KEY를 기반으로 하여 여러 실습을 진행해보고자 합니다.


## Open AI의  KEY 발급 안내

OpenAI의 API는 아래의 링크에서 발급 받을 수 있습니다. 

- OpenAI API 발급 : https://platform.openai.com/settings/organization/api-keys



<b>API를 사용하기 위해서는 ChatGPT의 유료 플랜 구독과는 별도로 유료 토큰을 구매하여 사용해야 합니다.</b>

- 본 실습 위해 미리 발급받은 실습용 OpenAI의 API를 제공해드릴 예정입니다.

- <b>제한되어 있는 key이기 때문에 본 실습 이외의 용도로 사용될 수 없음을 미리 안내드립니다.</b>


실습에서 사용된 함수 이외의 다른 자료는 아래의 [Open AI의 Docs]를 참고해주세요.

또한, Open AI API를 활용하여 다양한 프롬프트를 코드에서 활용해볼 수 있습니다.

- 참고자료 1 : docs (https://platform.openai.com/docs/models#models-overview)

- 참고자료 2 : Prompt examples (https://platform.openai.com/docs/examples)




In [1]:
# Lang chain을 위한 패키지 및 기타 패키지들을 설치합니다
!pip install langchain langchain-openai


[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# ChatOpenAI : OpenAI 사의 채팅 전용 LLM
from langchain_openai import ChatOpenAI


# GPT를 사용할 수 있는 API KEY 입력
api_key = "sk-gRqhsT2_U1YFOGYerzzIoOu1gpw0JmMnrXQosCx_4IT3BlbkFJRTMSFduGtHsnUrIP0Bsf3gfjjk4ucCPXv4Fw3lxXoA"

c:\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# 객체 생성
chat_model = ChatOpenAI(
    openai_api_key=api_key,
    model="gpt-4o-mini",
    temperature=0  # 위에서 선언한 API KEY  # model_name : 적용할 모델명
)  # temperature(창의성) : 사용할 샘플링 온도는 0~2 사이에서 선택.
# 0.8과 같은 높은 값을 출력을 더 무작위하게 만듦.
# 0.2와 같은 낮은 값은 출력을 더 집중되고 결정론적으로 만듦.
# max_token(최대 토큰수): 채팅 완성에서 생성할 토큰의 최대 개수

# 프롬프트 입력
chat_model.invoke("영어로 물품 거래(곡물)를 요청하는 비즈니스 메일을 작성해주세요.")

AIMessage(content="Subject: Inquiry for Grain Trade\n\nDear [Recipient's Name],\n\nI hope this message finds you well. My name is [Your Name], and I am [Your Position] at [Your Company Name]. We are a company specializing in [brief description of your company and its activities], and we are currently looking to expand our supply chain for high-quality grains.\n\nWe are interested in discussing potential trade opportunities for the following types of grains: [list specific grains you are interested in, e.g., wheat, corn, barley, etc.]. We would like to understand your pricing, availability, and any minimum order quantities you may have.\n\nAdditionally, if you could provide us with information regarding your shipping options and lead times, it would be greatly appreciated. We are looking to establish a long-term partnership and would be interested in any bulk purchase discounts you may offer.\n\nPlease let us know a convenient time for you to discuss this further. We look forward to you

In [4]:
prompt_template = "당신은 비즈니스 메일 작성 전문가입니다. 협상에 능통하고, 굉장히 예의바른 말투로 메일을 작성할 수 있습니다. "

In [5]:
prompt = (
    prompt_template + "영어로 물품 거래(곡물)를 요청하는 비즈니스 메일을 작성해주세요."
)

In [6]:
chat_model.invoke(prompt)

AIMessage(content="Subject: Inquiry for Grain Supply\n\nDear [Recipient's Name],\n\nI hope this message finds you well.\n\nMy name is [Your Name], and I am [Your Position] at [Your Company Name]. We are currently exploring opportunities to expand our supply chain and are interested in sourcing high-quality grain products.\n\nWe would greatly appreciate it if you could provide us with information regarding your available grain varieties, pricing, and any minimum order quantities. Additionally, if you could share your lead times and shipping options, it would be immensely helpful for our planning.\n\nWe are committed to establishing a mutually beneficial partnership and are eager to discuss how we can work together effectively. Please let us know a convenient time for you to discuss this further, or feel free to reply to this email with the requested information.\n\nThank you for considering our inquiry. I look forward to your prompt response.\n\nWarm regards,\n\n[Your Name]  \n[Your Pos

In [7]:
# 질의내용
question = "미국의 수도는 어디니?"

# 질의
print(f"[답변]: {chat_model.invoke(question).content}")

[답변]: 미국의 수도는 워싱턴 D.C.입니다.


## 프롬프트 템플릿 활용
PromptTemplate : 사용자의 입력 변수를 사용하여 완전한 프롬프트 문자열을 만드는 데 사용되는 템플릿

- template: 템플릿 문자열. 이 문자열 내에서 중괄호 {}는 변수.

- input_variables: 중괄호 안에 들어갈 변수의 이름을 리스트로 정의.

In [8]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

# 질문 템플릿 형식 정의
template = "{country}의 수도는 뭐야?"

# 템플릿 완성
prompt = PromptTemplate(template=template, input_variables=["country"])

LLMChain : 특정 PromptTemplate와 연결된 체인 객체를 생성

- prompt: 앞서 정의한 PromptTemplate 객체를 사용.

- llm: 언어 모델을 나타냄. 위에서 정의한 모델 사용

In [9]:
# 연결된 체인(Chain)객체 생성
llm_chain = LLMChain(prompt=prompt, llm=chat_model)

C:\Users\glory\AppData\Local\Temp\ipykernel_13428\1906816209.py:2: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(prompt=prompt, llm=chat_model)


In [10]:
# 체인을 통해 템플릿 프롬프트 실행
print(llm_chain.run(country="일본"))

C:\Users\glory\AppData\Local\Temp\ipykernel_13428\1629882101.py:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  print(llm_chain.run(country="일본"))


일본의 수도는 도쿄(東京)입니다. 도쿄는 일본의 정치, 경제, 문화의 중심지로, 많은 인구와 다양한 명소가 있는 대도시입니다.


In [11]:
# 여러 개의 입력을 한 번에 실행
input_list = [{"country": "중국"}, {"country": "프랑스"}, {"country": "미국"}]

print(llm_chain.batch(input_list))

[{'country': '중국', 'text': '중국의 수도는 베이징(北京)입니다.'}, {'country': '프랑스', 'text': '프랑스의 수도는 파리입니다.'}, {'country': '미국', 'text': '미국의 수도는 워싱턴 D.C.입니다.'}]


In [12]:
# text 키 값으로 결과가 반환됨 -> 반복문으로 출력

# input_list 에 대한 결과 반환
result = llm_chain.batch(input_list)

# 반복문으로 결과 출력
for res in result:
    print(res["text"].strip())

중국의 수도는 베이징(北京)입니다.
프랑스의 수도는 파리입니다.
미국의 수도는 워싱턴 D.C.입니다.


In [13]:
# LLMResult : 토큰 사용량과 종료 이유와 같은 유용한 생성 정보를 포함함
# input_list 에 대한 결과 반환
generated_result = llm_chain.generate(input_list)
print(generated_result)

generations=[[ChatGeneration(text='중국의 수도는 베이징(北京)입니다.', generation_info={'finish_reason': 'stop', 'logprobs': None}, message=AIMessage(content='중국의 수도는 베이징(北京)입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 15, 'total_tokens': 28, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CHfj6jM7w5JwzjJJiIPvQJLIGy5WU', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--872cfd92-0589-4ec2-963a-118fb5a10e53-0', usage_metadata={'input_tokens': 15, 'output_tokens': 13, 'total_tokens': 28, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}))], [ChatGeneration(text='프랑스의 수도는 파리입니다.', generation_i

In [14]:
# 답변 출력
generated_result.generations

[[ChatGeneration(text='중국의 수도는 베이징(北京)입니다.', generation_info={'finish_reason': 'stop', 'logprobs': None}, message=AIMessage(content='중국의 수도는 베이징(北京)입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 15, 'total_tokens': 28, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CHfj6jM7w5JwzjJJiIPvQJLIGy5WU', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--872cfd92-0589-4ec2-963a-118fb5a10e53-0', usage_metadata={'input_tokens': 15, 'output_tokens': 13, 'total_tokens': 28, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}))],
 [ChatGeneration(text='프랑스의 수도는 파리입니다.', generation_info={'finis

잠깐!! 아래의 형태와 같이 출력해주세요.

1 -> 중국의 수도는 베이징(北京)입니다.  
2 -> 프랑스의 수도는 파리입니다.  
3 -> 미국의 수도는 워싱턴 D.C.입니다.  

In [15]:
# 여기에 입력하세요!
for i, gen in enumerate(generated_result.generations):
    print(f"{i+1} -> {gen[0].text}")

1 -> 중국의 수도는 베이징(北京)입니다.
2 -> 프랑스의 수도는 파리입니다.
3 -> 미국의 수도는 워싱턴 D.C.입니다.


In [16]:
# 토큰 사용량 출력
generated_result.llm_output

{'token_usage': {'completion_tokens': 36,
  'prompt_tokens': 46,
  'total_tokens': 82,
  'completion_tokens_details': {'accepted_prediction_tokens': 0,
   'audio_tokens': 0,
   'reasoning_tokens': 0,
   'rejected_prediction_tokens': 0},
  'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}},
 'model_name': 'gpt-4o-mini',
 'system_fingerprint': 'fp_560af6e559'}

In [17]:
# run ID 출력
generated_result.run

[RunInfo(run_id=UUID('872cfd92-0589-4ec2-963a-118fb5a10e53')),
 RunInfo(run_id=UUID('d093544f-b2ff-45b2-88c1-3842c632dabf')),
 RunInfo(run_id=UUID('c379a9ce-9ab7-418d-b91d-924c745c7e5c'))]

In [18]:
# 답변 출력
for gen in generated_result.generations:
    print(gen[0].text.strip())

중국의 수도는 베이징(北京)입니다.
프랑스의 수도는 파리입니다.
미국의 수도는 워싱턴 D.C.입니다.


## 두 개 이상의 변수를 템플릿 안에 정의 

- input_variables를 활용하여 템플릿 구성

### 아래의 지역에 대하여, 시차를 구해주세요.

1. 파리 - 뉴욕
2. 서울 - 도쿄
3. 베이징 - 하와이

질문 예시 : _"서울과 파리의 시차는 몇 시간이야?"_

In [20]:
# 질문 템플릿 형식 정의
template = "{area1}와 {area2}의 시차는 몇 시간이야?"

# 템플릿 완성
prompt = PromptTemplate(template=template, input_variables=["area1", "area2"])

# 연결된 체인(Chain)객체 생성
llm_chain = LLMChain(prompt=prompt, llm=chat_model)

# run () : 체인 실행
print(llm_chain.run(area1="한국", area2="일본"))

한국과 일본은 같은 시간대에 속해 있습니다. 따라서 한국과 일본의 시차는 0시간입니다. 두 나라 모두 UTC+9 시간대를 사용하고 있습니다.


## 나만의 프롬프트 엔지니어링을 적용해 질의응답을 받을 수 있는 코드를 만들어주세요

In [ ]:
# 자유롭게 활용해보세요!